In [1]:
#######partially update a verblist with fragments of ngram######

In [1]:
import csv
import pandas as pd
from pyinflect import getAllInflections, getInflection

In [2]:
#set the mode to view the df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
%pprint

Pretty printing has been turned OFF


In [7]:
sent_comp = ['find', 'represent', 'recognize', 'guarantee', 'establish', 'deny', 'confirm', 'justify', 'announce', \
'verify', 'reveal', 'acknowledge', 'assert', 'assume', 'determine', 'discover', 'predict', 'regret', 'mean', 'allow', \
'expect', 'suggest', 'mention', 'doubt', 'realize', 'imagine', 'suppose', 'imply', 'observe']

In [8]:
len(sent_comp)

29

In [9]:
raw = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_Ngram_sentence_taking_verb2.xlsx')
raw_sub = raw[['Lemma', 'transitive', 'intransitive', 'totalcount']].set_index('Lemma')
raw_dict = raw_sub.to_dict(orient='index')

In [10]:
raw_sub.head()

,transitive,intransitive,totalcount
Lemma,,,
acknowledge,671410,139178,810588
acknowledged,119376,163468,282844
acknowledges,44825,22533,67358
allow,1162988,4703290,5866278
allowed,1598909,2126323,3725232


In [11]:
raw_dict['allow']

{'transitive': 1162988, 'intransitive': 4703290, 'totalcount': 5866278}

In [12]:
vv = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.23.xlsx')

In [54]:
#####compare old and new entries
vv_sub = vv[vv['Lemma'].isin(sent_comp)][['Lemma', 'google1950_45gram_intrans', 'google1950_45gram_trans']].set_index('Lemma')

In [55]:
vv_dict = vv_sub.to_dict(orient='index')

In [56]:
vv_sub.head()

,google1950_45gram_intrans,google1950_45gram_trans
Lemma,,
represent,24.85,75.15
recognize,38.17,61.83
guarantee,33.97,66.03
establish,26.82,73.18
deny,23.59,76.41


In [57]:
inflection_dict = {}
verbs_ending_ing = ['sing', 'swing']
for verb in sent_comp:
    inflection_dict[verb] = []
    inflections = getAllInflections(verb, pos_type='V')
    for k, v in inflections.items():
        v_list = list(v) ##tuple to list
        if not v_list[0].endswith('ing'): ###not going to keep a progressive form
            inflection_dict[verb].append(v_list[0]) ### if there are >1 variant for a given inflection, only keep 1st
        if verb in ['sing', 'swing']:
            inflection_dict[verb].append(verb)
    liszt = inflection_dict[verb] ##to drop duplicates, convert it into a set
    inflection_dict[verb] = list(set(liszt))

In [58]:
for k, v in vv_dict.items():
    #if k=='acknowledge':
    trans_count = 0
    intrans_count = 0
    total_count = 0
    wordforms = inflection_dict[k]
    for wordform in wordforms:
        trans_count += raw_dict[wordform]['transitive']
        intrans_count += raw_dict[wordform]['intransitive']
        total_count += raw_dict[wordform]['totalcount']
    trans_percent = round((trans_count/total_count)*100, 2)
    intrans_percent = round((intrans_count/total_count)*100, 2)
    v['intrans_new'] = intrans_percent
    v['trans_new'] = trans_percent
    v['trans_count'] = trans_count

In [26]:
# trans_c = 671410+119376+44825
# intrans_c = 139178 + 163468 + 22533 
# total_c = trans_c + intrans_c

In [40]:
(trans_c/total_c)*100 

71.98640580983641

In [59]:
vv_update = pd.DataFrame.from_dict(vv_dict, orient='index')

In [60]:
vv_update

,google1950_45gram_intrans,google1950_45gram_trans,intrans_new,trans_new,trans_count
represent,24.85,75.15,24.85,75.15,5298587
recognize,38.17,61.83,37.99,62.01,2275421
guarantee,33.97,66.03,33.95,66.05,461975
establish,26.82,73.18,26.82,73.18,5595342
deny,23.59,76.41,23.41,76.59,1272818
confirm,28.35,71.65,28.34,71.66,1059070
justify,17.05,82.95,17.05,82.95,1434456
announce,38.36,61.64,38.32,61.68,560123
verify,31.42,68.58,31.40,68.60,385623
reveal,34.73,65.27,34.72,65.28,1513256


In [51]:
vv_dict['represent']

{'google1950_45gram_intrans': 24.85, 'google1950_45gram_trans': 75.15, 'intrans_new': 24.85, 'trans_new': 75.15, 'total_count': 7050787}

In [61]:
for k, v in vv_dict.items():
    diff = v['trans_new'] - v['google1950_45gram_trans']
    print (k, diff*v['trans_count'])

represent 0.0
recognize 409575.77999999933
guarantee 9239.499999998161
establish 0.0
deny 229107.2400000087
confirm 10590.699999990367
justify 0.0
announce 22404.91999999952
verify 7712.459999998466
reveal 15132.560000007741
acknowledge 167122.1999999905
assert 125144.00000000012
find 559260.7500000212
assume 362485.9799999994
determine 62332.7599999876
discover 74429.75999999841
predict 14988.399999997018
regret 5741.079999999878
mean 131980.25000000186
allow 0.0
expect 0.0
suggest 205452.38000000085
mention 0.0
doubt 27769.68000000105
realize 359864.00999999815
imagine 40344.639999998115
suppose 83631.05999999326
imply 66164.20000000094
observe 34125.220000005334


In [23]:
vv_update

,google1950_45gram_intrans,google1950_45gram_trans,intrans_new,trans_new
represent,24.85,75.15,24.85,75.15
recognize,38.17,61.83,37.99,62.01
guarantee,33.97,66.03,33.95,66.05
establish,26.82,73.18,26.82,73.18
deny,23.59,76.41,23.41,76.59
confirm,28.35,71.65,28.34,71.66
justify,17.05,82.95,17.05,82.95
announce,38.36,61.64,38.32,61.68
verify,31.42,68.58,31.40,68.60
reveal,34.73,65.27,34.72,65.28


In [6]:
#raw.head()

Unnamed: 0  transitive  intransitive  \
0   acknowledge      670651        139937   
1  acknowledged      119376        163468   
2  acknowledges       44825         22533   
3         allow     1162981       4703297   
4       allowed     1598899       2126333   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      